# This is the notebook to remake individual season raw stats database



In [1]:
#import our libraries
import sys
import time
import psycopg2
import pandas as pd
import numpy as np
import urllib2
import re
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database



In [2]:
def make_it_lower(array):
    
    new_array = []
    for item in array:
        new_array.append(item.lower())
        
    return new_array

In [3]:
def bs_column_names(tag):
    return tag.name == 'th' and tag.has_attr('width') 

In [4]:
def get_column_names(namedata):
    
    column_names = []
    for name in namedata.find_all(bs_column_names):
        try:
            match = re.search('([A-Z3].*[A-Z])', str(name))
            #if name.get_text() not in column_names: 
            if match.group(0).strip() not in column_names: 
                column_names.append(match.group(0).strip())
        except:
            a = 1
            
    return column_names

In [5]:
def bs_column_data(tag):
    return tag.name == 'td' and not tag.has_attr('style')

In [6]:
def get_column_data(datadata):

    column_data = []
    for data in datadata.find_all(bs_column_data):
        try:
            match = re.search('[0-9]*[0-9\-]*[0-9\-]*[0-9\-]', str(data))
            column_data.append(match.group(0).strip())
        except:
            a = 1 
            
    return column_data

In [7]:
def clean_player(mydict):
    mykeys = mydict.keys()
    myvals = mydict.values()

    for ii in np.arange(len(mykeys)):
        match_key = re.search('(.*?)-(.)', mykeys[ii])
        match_value = re.search('(.*?)-(.*)', myvals[ii])
        
        #### figure out why the 3 pointer isnt tranlating 
        
        if match_key is not None:
            checking = re.search('^3', match_key.group(1))
            if checking is None:
                thatkey = match_key.group(1)
            else:
                thatkey = match_key.group(1)
                thatkey = thatkey.replace('3','t')
                
            #print match_key.group(1), match_value.group(1)
            mydict[thatkey] = match_value.group(1)
            newkey = thatkey
            newkey = newkey.replace(newkey[-1],match_key.group(2))
            #print newkey, match_value.group(2) 
            mydict[newkey] = match_value.group(2) 
       
    return mydict

In [8]:
def build_player(bit1, alldict, team, data_columns, gameid):
 
    #print bit1.prettify()
    #team1_columns = get_column_names(bit1)
    #print team1_columns
    column_data = get_column_data(bit1)
    #print column_data
    matches = re.findall(', ([A-Z])', str(bit1))
    #print matches


    for player in bit1.find_all('a'):

        #start a dictionaries
        team1_dict1 = {'team_name':team, 'player':'', 'player_pos':'', 'player_url':'', 'game_id':str(gameid)}
        team1_dict2 = {}

        #start filling dictionaries
        team1_dict1['player_url'] = player['href']
        team1_dict1['player'] = player.get_text().replace(r'\n', '').strip()
        team1_dict1['player_pos'] = matches[0]
        del matches[0]
        #print team1_dict1
   
        for column in data_columns:
            #print column
            #print column_data[0]
            team1_dict2[column] = column_data[0]
            del column_data[0]
        #print team1_dict2     

        #combine the two dictionaries
        finaldict = team1_dict1.copy()
        finaldict.update(team1_dict2)
        finaldict = clean_player(finaldict)
        #print finaldict
        #print ''
        alldict.append(finaldict)
    

        #print alldict[0]['tpa'], alldict[0]['tpm']
        #print alldict
        #print ''
    return alldict

In [9]:
def build_box_score(soup, gameid):

    if gameid > 323140002:
        inds = [1,3,5,11,13,15]
    else:
        inds = [1,3,7,13,15,19]
    
    alldict1 = []
    try:
        box_score = soup.body.div.table

        #begin with the first team
        bit1 = box_score.contents[inds[0]]
        team = bit1.th.get_text()
        team = team.replace(r'\n', '')
        team = team.strip()
        data_columns = get_column_names(bit1)
        data_columns = make_it_lower(data_columns)   
        alldict1 = build_player(box_score.contents[inds[1]], alldict1, team, data_columns, gameid)
        #print alldict1
        #print '**************'
        alldict1 = build_player(box_score.contents[inds[2]], alldict1, team, data_columns, gameid)
        
        #now do the second team
        bit1 = box_score.contents[inds[3]]
        team = bit1.th.get_text()
        team = team.replace(r'\n', '')
        team = team.strip()        
        data_columns = get_column_names(bit1)
        data_columns = make_it_lower(data_columns)
        alldict1 = build_player(box_score.contents[inds[4]], alldict1, team, data_columns, gameid)
        alldict1 = build_player(box_score.contents[inds[5]], alldict1, team, data_columns, gameid)
    except:
        alldict1 = []
        
    return alldict1

In [10]:
def get_med_dir(my_con, boxscore_table_name, id):
    '''
    function to return a directory folder based on 
    specified user inputs and knowledge of the games db.
    
    my_con: connection to Postgres server
    boxscore_table_name: name of table on Postgres server to query
    id: unique ESPN game id to find month and year of
    '''
    
    
    sql_query = " SELECT year, month FROM %s WHERE id=%s; " % (boxscore_table_name, id)
    #print sql_query
    try:
        from_sql_query = pd.read_sql_query(sql_query, my_con)
        #print from_sql_query.head(5)
    except:
        print '  games table, %s, does not exist' % boxscore_table_name

    month = from_sql_query['month'][0]
    year = from_sql_query['year'][0]
    if int(month) > 7:
        med_dir = str(year) + '-' + str(int(year)+1) + '/'
    else:
        med_dir = str(int(year)-1) + '-' + str(year) + '/'

    return med_dir

In [11]:
def create_snippet(file_in, file_out=None):
    
    
    if file_out is None:
        file_out = 'temp.txt'
    else:
        file_out = str(file_out)
    
    #print 'File in:', file_in
    #print 'File out:', file_out
    
    try:
        #read in entire web page
        target = open(file_in, 'r')
        text = target.read()
        target.close()

        #apply BS
        soup = BeautifulSoup(text, 'lxml')
        box_score = soup.div(id='my-players-table')

        #read the file back in
        target = open(file_out, 'w')
        text = target.write(str(box_score))
        target.close()

        #reapply BS - makes the structure a bit more managable 
        #  and hopefully consistent :|
        target = open(file_out, 'r')
        text = target.read()
        target.close()
        soup = BeautifulSoup(text, 'lxml')

        #resave file with better structure
        #target = open(file_out, 'w')
        #text = target.write(str(soup.prettify()))
        #target.close()
    except:
        soup = None
        
    return soup

In [12]:
def start_games_db(dbname, username):
    print '  Firing up the data base.'
    
    engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
    print '    DB url:', engine.url
    db_exist = database_exists(engine.url)
    if not db_exist:
        create_database(engine.url)
    print '    DB exists? %s' % db_exist
    
    out_dict = {'dbname':dbname, 'username':username, 
                'exists':db_exist, 'engine_url':engine.url, 'engine':engine}
    
    return out_dict

In [13]:
def query_boxscore_db(my_con, boxscore_table_name, year):
    print '    Now getting games for which to find stats.'

    #determine appropriate SQL query 
    if year == '1516':
        ending = "400809203 AND 400863476; "
    elif year == '1415':
        ending = "400585667 AND 400788981; "
    elif year == '1314':
        ending = "400496833 AND 400551235; "

    elif year == '1213':
        ending = "323140002 AND 330980097; "

    else:
        print 'WARNING!!! This year has not been set up yet'
        sys.exit(0)
    sql_query = " SELECT DISTINCT(id) FROM %s WHERE in_hand='%s' and id BETWEEN " % (boxscore_table_name, 'yes')
    sql_query = sql_query + ending

    try:
        from_sql_query = pd.read_sql_query(sql_query, my_con)
    except:
        print '  games table, %s, does not exist' % boxscore_table_name

    return from_sql_query

In [14]:
def make_gamestats_db(reset, dataframe, db_connect, stats_table):
    
    #print reset
    #print dataframe
    #print stats_table
    #print db_connect
    
    if reset == 1:
        my_if_exists = 'replace'
    else:
        my_if_exists = 'append'
    
    #print my_if_exists
    dataframe.to_sql(stats_table, db_connect['engine'], if_exists=my_if_exists)

    reset = 0 #now it will append instead of replacing
    return reset

In [15]:
def main(remake_db=False, year=None):
    
    dbname = 'ncaa_mbb_db'
    username = 'smaug'

    boxscore_dir = 'boxscore_pages/'
    boxscore_file = 'ncaa_mbb_boxscore_DDDDDDDDD.txt'
    boxscore_table_name = 'games'

    
    if year is None:
        year = '1314'
    else:
        if len(year) != 4:
            print 'WARNING!! Year input must be string of form Y1Y1Y2Y2'
            sys.exit(0)
        else:
            year = str(year)    
    
    #set the table name based on the year
    stats_table_name = 'stats' + year
    print stats_table_name

    
    reset = 1
    if remake_db:
        print '  Now remaking the statistics database.'
        
        #fire up the database engine
        db_connect = start_games_db(dbname, username)
        #print db_engine
        
        #get a connection to the database
        my_con = None
        my_con = psycopg2.connect(database=dbname, user=username)
        #print my_con

        
        #query the boxscore database
        games_to_get = query_boxscore_db(my_con, boxscore_table_name, year)
        print len(games_to_get)

        cnt = 1
        alldict = [] #this is an intial value!!!!
        team = 'unknown'#this is an intial value!!!!
        for game_to_get in games_to_get['id']:
            print game_to_get

            med_dir = get_med_dir(my_con, boxscore_table_name, game_to_get)
            this_file = (boxscore_dir + med_dir + 
                        boxscore_file.replace('DDDDDDDDD', str(game_to_get)))
            this_game = create_snippet(this_file)
            #print this_game
            
            if this_game is not None:
                my_boxscore = build_box_score(this_game, game_to_get)
                if my_boxscore != []:
                    df_team1 = pd.DataFrame.from_records(my_boxscore)

                    df_team1 = df_team1.drop('3pm-a', 1)
                    df_team1 = df_team1.drop('fgm-a', 1)
                    df_team1 = df_team1.drop('ftm-a', 1)

                    home_away = []
                    away_score = 0
                    home_score = 0
                    for ii in np.arange(len(df_team1.loc[:,'team_name'])):
                        chk = df_team1.loc[ii,'team_name'] == df_team1.loc[0,'team_name']
                        if chk:
                            home_away.append('A')
                            away_score = away_score + int(df_team1.loc[ii,'pts'])
                        else:
                            home_away.append('H')
                            home_score = home_score + int(df_team1.loc[ii,'pts'])

                    df_team1['ha'] = home_away
                    if away_score > home_score:
                        wls = ['w', 'l']
                    else:
                        wls = ['l', 'w']

                    win_loss = []
                    for ii in np.arange(len(df_team1.loc[:,'team_name'])):
                        chk = df_team1.loc[ii,'team_name'] == df_team1.loc[0,'team_name']
                        if chk:
                            win_loss.append(wls[0])
                        else:
                            win_loss.append(wls[1])

                    df_team1['wl'] = win_loss

                    #df_team1.to_csv('example1.csv')
                    #df_team2 = pd.DataFrame.from_csv('example1.csv')
                    #print df_team2

                    #print df_team1
                    reset = make_gamestats_db(reset, df_team1, 
                                              db_connect, stats_table_name)

                #if cnt >=3:
                #    sys.exit(0)
                #cnt = cnt + 1


In [16]:
# boilerplate to execute call to main() function
if __name__ == '__main__':
    main(remake_db=True, year='1516')

stats1516
  Now remaking the statistics database.
  Firing up the data base.
    DB url: postgres://smaug@localhost/ncaa_mbb_db
    DB exists? True
    Now getting games for which to find stats.
5328
400840012
400845568
400830177
400830215
400809384
400820706
400841080
400830673
400845003
400827779
400809341
400825726
400845520
400816875
400841584
400830597
400830504
400814222
400840959
400841195
400845571
400818912
400830802
400830884
400816861
400847041
400827471
400817483
400817627
400824553
400835248
400816843
400846753
400830403
400843796
400844954
400847063
400844899
400839208
400817147
400842160
400817201
400818905
400833337
400839751
400830632
400830586
400830507
400816867
400844911
400845090
400820618
400818729
400830466
400830412
400830253
400841619
400840825
400830402
400841198
400833123
400840209
400830376
400813824
400816935
400843706
400827704
400814225
400843054
400824538
400840949
400841293
400817133
400842004
400845074
400847070
400843659
400831554
400820723
400835678
